<a href="https://colab.research.google.com/github/drstannwoji2019/ML_Projects/blob/main/DeepLearning_GhanaRem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load Dataset
file_path = "/Remittances_Ghana.csv"  # Update with correct file path
df = pd.read_csv(file_path)

# Convert 'Date' column to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'], format='%Y')
df.set_index('Date', inplace=True)

# Normalize the target variable
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df[['Rem_Ghana']])

# Split into training and testing sets (80% train, 20% test)
train_size = int(len(df_scaled) * 0.8)
train_data, test_data = df_scaled[:train_size], df_scaled[train_size:]

# Function to create sequences for time series forecasting
def create_sequences(data, time_step=3):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step)])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

# Define lookback window
time_step = 3
X_train, y_train = create_sequences(train_data, time_step)
X_test, y_test = create_sequences(test_data, time_step)

# Reshape data for deep learning models
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Function to build and train a model
def build_and_train_model(model_type, X_train, y_train, X_test, y_test, epochs=100, batch_size=1):
    model = Sequential()
    if model_type == "RNN":
        model.add(SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    elif model_type == "LSTM":
        model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    elif model_type == "GRU":
        model.add(GRU(50, activation='relu', input_shape=(X_train.shape[1], 1)))

    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict on test data
    predictions = model.predict(X_test)
    return model, predictions

# Train RNN, LSTM, and GRU models
rnn_model, rnn_predictions = build_and_train_model("RNN", X_train, y_train, X_test, y_test)
lstm_model, lstm_predictions = build_and_train_model("LSTM", X_train, y_train, X_test, y_test)
gru_model, gru_predictions = build_and_train_model("GRU", X_train, y_train, X_test, y_test)

# Inverse transform predictions to original scale
rnn_predictions = scaler.inverse_transform(rnn_predictions)
lstm_predictions = scaler.inverse_transform(lstm_predictions)
gru_predictions = scaler.inverse_transform(gru_predictions)

# Inverse transform actual values
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Function to evaluate model performance
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# Evaluate models
rnn_mae, rnn_rmse = evaluate_model(y_test_actual, rnn_predictions)
lstm_mae, lstm_rmse = evaluate_model(y_test_actual, lstm_predictions)
gru_mae, gru_rmse = evaluate_model(y_test_actual, gru_predictions)

# Print results
print(f"RNN - MAE: {rnn_mae:.2f}, RMSE: {rnn_rmse:.2f}")
print(f"LSTM - MAE: {lstm_mae:.2f}, RMSE: {lstm_rmse:.2f}")
print(f"GRU - MAE: {gru_mae:.2f}, RMSE: {gru_rmse:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
RNN - MAE: 581871593.00, RMSE: 581871593.00
LSTM - MAE: 866617065.00, RMSE: 866617065.00
GRU - MAE: 875107561.00, RMSE: 875107561.00


In [7]:
import pandas as pd

# Create a DataFrame to store and display the results
metrics_data = {
    "Model": ["RNN", "LSTM", "GRU"],
    "MAE": [rnn_mae, lstm_mae, gru_mae],
    "RMSE": [rnn_rmse, lstm_rmse, gru_rmse]
}

metrics_df = pd.DataFrame(metrics_data)

# Print the table
print(metrics_df)


  Model          MAE         RMSE
0   RNN  581871593.0  581871593.0
1  LSTM  866617065.0  866617065.0
2   GRU  875107561.0  875107561.0
